### Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from ETGEMs_function_ETG import *
import pandas as pd
from ppmcot import *
from strategy_pyf import *

In [2]:
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

### Data initialization

In [3]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './HSEC0916-g0.txt'
reaction_g0_file_original1 = './HSEC1017-g0.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './HSEC0916-lnC.txt'
metabolites_lnC_file_original1 = './HSEC1017-lnC.txt'
#Get Model from local file
model_file_original0 = './HSEC0916.xml'
model_file_original1 = './HSEC1017.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './HSEC0916-kcat-MW.csv'
reaction_kcat_MW_file_original1 = './HSEC1017-kcat-MW.csv'

In [4]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

### Strain reconstruction

In [5]:
model0['ub_list']['ACALD']=0
model0['lb_list']['ACALD']=0
model0['ub_list']['ACALD_reverse']=0
model0['lb_list']['ACALD_reverse']=0
model0['ub_list']['ALCD2x']=0
model0['lb_list']['ALCD2x']=0
model0['ub_list']['ALCD2x_reverse']=0
model0['lb_list']['ALCD2x_reverse']=0
model0['ub_list']['LDH_D']=0
model0['lb_list']['LDH_D']=0
model0['ub_list']['FRD2']=0
model0['lb_list']['FRD2']=0
model0['ub_list']['FRD3']=0
model0['lb_list']['FRD3']=0
model0['ub_list']['PTAr']=0
model0['lb_list']['PTAr']=0
model0['ub_list']['PTAr_reverse']=0
model0['lb_list']['PTAr_reverse']=0
model0['ub_list']['PTA2']=0
model0['lb_list']['PTA2']=0
model0['ub_list']['PFL']=0
model0['lb_list']['PFL']=0
model0['ub_list']['OBTFL']=0
model0['lb_list']['OBTFL']=0
model0['ub_list']['ACKr']=0
model0['lb_list']['ACKr']=0
model0['ub_list']['PTAr']=0
model0['lb_list']['PTAr']=0
model0['ub_list']['PTA2']=0
model0['lb_list']['PTA2']=0

In [6]:
model1['ub_list']['ACGAptspp']=0
model1['lb_list']['ACGAptspp']=0
model1['ub_list']['ACMANAptspp']=0
model1['lb_list']['ACMANAptspp']=0
model1['ub_list']['ACMUMptspp']=0
model1['lb_list']['ACMUMptspp']=0
model1['ub_list']['ARBTptspp']=0
model1['lb_list']['ARBTptspp']=0
model1['ub_list']['ASCBptspp']=0
model1['lb_list']['ASCBptspp']=0
model1['ub_list']['CHTBSptspp']=0
model1['lb_list']['CHTBSptspp']=0
model1['ub_list']['DHAPT']=0
model1['lb_list']['DHAPT']=0
model1['ub_list']['FRUpts2pp']=0
model1['lb_list']['FRUpts2pp']=0
model1['ub_list']['FRUptspp']=0
model1['lb_list']['FRUptspp']=0
model1['ub_list']['GALTptspp']=0
model1['lb_list']['GALTptspp']=0
model1['ub_list']['GAMptspp']=0
model1['lb_list']['GAMptspp']=0
model1['ub_list']['GLCptspp']=0
model1['lb_list']['GLCptspp']=0
model1['ub_list']['MALTptspp']=0
model1['lb_list']['MALTptspp']=0
model1['ub_list']['MANptspp']=0
model1['lb_list']['MANptspp']=0
model1['ub_list']['MNLptspp']=0
model1['lb_list']['MNLptspp']=0
model1['ub_list']['SBTptspp']=0
model1['lb_list']['SBTptspp']=0
model1['ub_list']['SUCptspp']=0
model1['lb_list']['SUCptspp']=0
model1['ub_list']['TREptspp']=0
model1['lb_list']['TREptspp']=0
model1['ub_list']['HEX1']=0
model1['lb_list']['HEX1']=0
model1['ub_list']['EX_glc__D_e_reverse']=0
model1['lb_list']['BIOMASS_Ec_iML1515_core_75p37M'] = 0.457*0.991 #0.19

### Analyze the biosynthesis strategy

In [7]:
#If you want to simulate the production with the unit of g/L, the real molar masses of exchange metabolites and carbon sources should be input, and the concentrations of exchange metabolites and carbon sources should be input with the unit of g/L
#If you want to simulate the production with the unit of mM, the molar masses of exchange metabolites and carbon sources should be input with the same value, and the concentrations of exchange metabolites and carbon sources should be input with the unit of mM

#model_list: the list of models calculated by the function of Get_Concretemodel_Need_Data
#target_list: the double list consist of the list of target reactions of each model, where the first reaction of each target reaction list is biomass biosynthesis reaction and the second is the reaction whose flux need to be maximized or minimized
#growth_list: the list of minimize growth rates in the function of MDF_Calculation
#growth_ratio: the list of ratios of lower bounds of growth rates to maximum growth rates
#parameter_list: the list of the total number of enzyme ratios, the upper bounds for substrate input reaction fluxes and the maximum values minus the minimum values of reaction thermodynamic driving forces
#substrate_environment: the list of the carbon source uptake reactions and the initial carbon source concentrations in environment
#metabolites_monitored: the list of metabolites delivered by the strains and the product
#metabolites_mass: the dictionary whose keys are the metabolites in metabolites_monitored and values are the molar masses of the metabolites
#concentrations_initial: the dictionary whose keys are the metabolites in metabolites_monitored and values are the initial concentrations of the metabolites in environment
#reactions_monitored: the list of dictionaries whose keys are the reactions of the metabolite uptake or release in metabolites_monitored and values are the ratios of lower bounds of the reaction fluxes to the max reaction fluxes
#substrate_mass: the list of the molar masses of the carbon sources
#t: the number of the time points for mono-cultivation
#culture_condition: the list of the inoculum ratio of the straints and time points for co-cultivation
#target_metabolite: the product id in the metabolic network 
#map_coefficient: the list of mapping constants of the instantaneous fluxes corresponding to the mean fluxes for the mono strains
#constraints: the constraints used in the simulation, where 'kinetics and thermodynamics' consist of FBA, kinetic and thermodynamic constraints, 'kinetics' consist of FBA and kinetic constraints, 'thermodynamics' consist of FBA and thermodynamic constraints and 'FBA' consist of FBA constraint
    
model_list=[model0, model1]
target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_ibutoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_isobt_e']]
growth_list=[0.01, 0.01]
growth_ratio=[0.965, 0.989]
parameter_list=[[0.19, 25, 1249], [0.19, 10, 1249]]
substrate_environment=[['EX_glc__D_e_reverse',25],['EX_xyl__D_e_reverse',10]]
metabolites_monitored=['ibutoh_e', 'isobt_e']
metabolites_mass={'ibutoh_e': 74, 'isobt_e': 144}
concentrations_initial={'ibutoh_e': 0.007, 'isobt_e': 0}
reactions_monitored=[{'EX_ibutoh_e': 0}, {'EX_ibutoh_e_reverse': 0, 'EX_isobt_e': 0}]
substrate_mass=[180,150]
t=10
culture_condition=[[1/5, 4/5], 10]
target_metabolite='isobt_e'
map_coefficient=[2, 1.88]
constraints='thermodynamics'

strategy_pyf(model_list, target_list, growth_list, growth_ratio, parameter_list, substrate_environment,  metabolites_monitored, metabolites_mass, concentrations_initial, reactions_monitored, substrate_mass, t, culture_condition, target_metabolite, map_coefficient, constraints)

After 1 time point, the concentration of ibutoh_e is 0.020171254652433822 g/L or mM
After 1 time point, the concentration of isobt_e is 0.0025676211891551503 g/L or mM
After 2 time point, the concentration of ibutoh_e is 0.03085978048790125 g/L or mM
After 2 time point, the concentration of isobt_e is 0.009966498454569234 g/L or mM
After 3 time point, the concentration of ibutoh_e is 0.03953356195748695 g/L or mM
After 3 time point, the concentration of isobt_e is 0.02128595935088829 g/L or mM
After 4 time point, the concentration of ibutoh_e is 0.046572370315185935 g/L or mM
After 4 time point, the concentration of isobt_e is 0.03578698954601395 g/L or mM
After 5 time point, the concentration of ibutoh_e is 0.05263037906093379 g/L or mM
After 5 time point, the concentration of isobt_e is 0.05219660276980125 g/L or mM
After 6 time point, the concentration of ibutoh_e is 0.05868838156069581 g/L or mM
After 6 time point, the concentration of isobt_e is 0.06860622814793936 g/L or mM
After

After 1 time point, the concentration of ibutoh_e is 0.02022659605853648 g/L or mM
After 1 time point, the concentration of isobt_e is 0.0025784095134793314 g/L or mM
After 2 time point, the concentration of ibutoh_e is 0.030949556202340044 g/L or mM
After 2 time point, the concentration of isobt_e is 0.010028759185357856 g/L or mM
After 3 time point, the concentration of ibutoh_e is 0.0396427886664493 g/L or mM
After 3 time point, the concentration of isobt_e is 0.021428849206911793 g/L or mM
After 4 time point, the concentration of ibutoh_e is 0.04669049654388959 g/L or mM
After 4 time point, the concentration of isobt_e is 0.036031041126848566 g/L or mM
After 5 time point, the concentration of ibutoh_e is 0.05277395910804273 g/L or mM
After 5 time point, the concentration of isobt_e is 0.05250960230507387 g/L or mM
After 6 time point, the concentration of ibutoh_e is 0.05885742167201572 g/L or mM
After 6 time point, the concentration of isobt_e is 0.06898816348364975 g/L or mM
After

{0: {'growth_ratio': [0.059627779942520984, -5.96247619003704],
  'CS_utilization': 0},
 1: {'growth_ratio': [0.6667662073428724, -66.67336152711661],
  'CS_utilization': 0.48994652838494385}}